In [2]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

In [16]:
#MNIST数据集相关的数据
INPUT_NODE = 784 #输入层的节点数。对于MNIST数据集，这个就是等于图片的像素
OUTPUT_NODE = 10 #输出层的节点属于。这个等于类别的数目，因为在MNIST数据集中需要区分的是0~9这十个数字，所以这里输出层的节点数为10

#配置神经网络的参数
LAYER1_NODE = 500 #隐藏层节点数，这里使用只有一个隐藏层的网络结构作为样例，这个隐藏层有500个节点
BATCH_SIZE = 100 #一个训练batch中的训练数据的个数，数据越小时，训练过程越接近随机梯度下降，数字越大，训练越接近梯度下降

LEARNING_RATE_BASE = 0.8 #基础的学习率
LEARNING_RATE_DECAY = 0.99 #学习率的衰减率
REGULARIZATION_RATE = 0.0001 #描述模型复杂度的正则化项在损失函数中的系数

TRANING_STEPS = 3000 #训练轮数
MOVING_AVERAGE_DECAY = 0.99 #滑动平均衰减率

#辅助函数，计算神经网络的钱箱传播结果
def inference(input_tensor,avg_class,weight1,biase1,weight2,biase2):
    if avg_class == None:
        layer1 = tf.nn.relu(tf.matmul(input_tensor,weight1) + biase1)
        return tf.matmul(layer1,weight2) + biase2
    else:
        layer1 = tf.nn.relu(tf.matmul(input_tensor,avg_class.average(weight1)) + avg_class.average(biase1))
        return tf.matmul(layer1,avg_class.average(weight2)) + avg_class.average(biase2)

#训练模型的过程

def train(mnist):
    x = tf.placeholder(tf.float32,[None,INPUT_NODE],name='x-input')
    y_ = tf.placeholder(tf.float32,[None,OUTPUT_NODE],name='y-output')
    
    #生成隐藏层的参数
    weight1 = tf.Variable(tf.truncated_normal([INPUT_NODE,LAYER1_NODE],stddev=0.1))
    biase1 = tf.Variable(tf.constant(0.1,shape=[LAYER1_NODE]))
    
    #生成输出层的参数
    weight2 = tf.Variable(tf.truncated_normal([LAYER1_NODE,OUTPUT_NODE],stddev=0.1))
    biase2 = tf.Variable(tf.constant(0.1,shape=[OUTPUT_NODE]))
    
    #所以函数不会使用参数的滑动平均值
    y = inference(x,None,weight1,biase1,weight2,biase2)
    
    #定义存储训练轮数的变量,这个变量不需要计算滑动平均值，所以这里指定为不可训练的变量
    global_step = tf.Variable(0,trainable=False)
    
    #给定滑动平均衰减率和训练轮数的变量，初始化滑动平均类
    variable_averages = tf.train.ExponentialMovingAverage(MOVING_AVERAGE_DECAY,global_step)
    #在所有代表神经网络参数的变量上使用滑动平均
    variables_averages_op = variable_averages.apply(tf.trainable_variables())
    #计算使用滑动平均之后的前向传播结果
    average_y = inference(x,variable_averages,weight1,biase1,weight2,biase2)
    
    #损失函数
    cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=y,labels=tf.argmax(y_,1))
    #计算在当前所有batch中所有样例的交叉熵平均值
    cross_entropy_mean = tf.reduce_mean(cross_entropy)
    
    #计算L2正则化损失函数
    regularizer = tf.contrib.layers.l2_regularizer(REGULARIZATION_RATE)
    #计算模型的正则化损失
    regularization = regularizer(weight1) + regularizer(weight2)
    loss = cross_entropy_mean + regularization
    #设置指数衰减的学习率
    learning_rate = tf.train.exponential_decay(LEARNING_RATE_BASE,global_step,mnist.train.num_examples/BATCH_SIZE,LEARNING_RATE_DECAY)
    
    train_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss,global_step = global_step)
    
    #反向传播更新神经网络的参数
    with tf.control_dependencies([train_step,variables_averages_op]):train_op = tf.no_op(name='train')

    correct_prediction = tf.equal(tf.argmax(y,1),tf.argmax(y_,1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))
    
    with tf.Session() as sess:
        #初始化变量
        tf.global_variables_initializer().run()
        #验证数据集
        validate_feed = {x:mnist.validation.images,y_:mnist.validation.labels}
        #测试数据集
        test_feed = {x:mnist.test.images,y_:mnist.test.labels}
        
        for epoch in range(TRANING_STEPS):
            xs,ys = mnist.train.next_batch(BATCH_SIZE)
            sess.run(train_step,feed_dict={x:xs,y_:ys})
            if epoch % 100 == 0:
                validate_acc = sess.run(accuracy,feed_dict=validate_feed)
                print('After %d training step(s),validation accuracy,using average model is%g' %(epoch,validate_acc))
            test_acc = sess.run(accuracy,feed_dict=test_feed)
        print('After %d training steps(s),test accuracy using average, model is %g'%(TRANING_STEPS,test_acc))
def main(argv=None):
    mnist = input_data.read_data_sets('/Users/wangyingbo/Downloads/mnist',one_hot=True)
    train(mnist)
if __name__ == '__main__':
    tf.app.run()

Extracting /Users/wangyingbo/Downloads/mnist/train-images-idx3-ubyte.gz
Extracting /Users/wangyingbo/Downloads/mnist/train-labels-idx1-ubyte.gz
Extracting /Users/wangyingbo/Downloads/mnist/t10k-images-idx3-ubyte.gz
Extracting /Users/wangyingbo/Downloads/mnist/t10k-labels-idx1-ubyte.gz
After 0 training step(s),validation accuracy,using average model is0.1172
After 100 training step(s),validation accuracy,using average model is0.9314
After 200 training step(s),validation accuracy,using average model is0.9444
After 300 training step(s),validation accuracy,using average model is0.9574
After 400 training step(s),validation accuracy,using average model is0.9592
After 500 training step(s),validation accuracy,using average model is0.969
After 600 training step(s),validation accuracy,using average model is0.9666
After 700 training step(s),validation accuracy,using average model is0.967
After 800 training step(s),validation accuracy,using average model is0.9676
After 900 training step(s),validat

SystemExit: 

/Users/wangyingbo/anaconda3/envs/tensorflow/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3304: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
